## The Flying Salesman
###  - A Time-dependent Traveling Salesman Problem

Reference: https://github.com/rhiever/Data-Analysis-and-Machine-Learning-Projects/tree/master/optimal-road-trip 

#### The scenario - 
The traveling salesman has to visit a number of destinations by air. Unlike the traditional TSP, as the flight ticket price changes, the edge costs varies depending on their positions in the tour. Presented below is a solution to this problem using a genetic algorithm.

#### The inputs -
1. a list of destinations
2. a list representing the number of nights to stay at each destination


In [22]:
from array import *
import random

destinations = ['dest_a', 'dest_b', 'dest_c', 'dest_d', 'dest_e', 'dest_f', 'dest_g', 'dest_h', 'dest_i', 'dest_j',
                'dest_k', 'dest_l', 'dest_m', 'dest_n', 'dest_o']
num_of_nights_to_stay = [3, 2, 1, 4, 2, 4, 2, 5, 1, 1, 2, 2, 1, 4, 3]
total_nights = sum(num_of_nights_to_stay)

print(total_nights)

37


In [23]:
# The most time-consuming part of the algorithm. We want one-way flight fares between each pair of destinations for 
#   every day through the duration of the entire tour. In practice, this will be done by web scraping. 
#   Here we generate some random data to manipulate.

# a 3-d array, flight_fare[x][y][z] is the flight price from destinations[x] to destinations[y] on day z
flight_fares = []

for x in range(0, len(destinations)):
    
    from_x = []
    
    for y in range(0, len(destinations)):
        
        from_x_to_y = []
        # to be more realistic
        # assume that the flight price from x to y varies within 40% of an "average" value over the days
        # further assume that the average is a multiple of 10 between 50 and 250
        from_x_to_y_average = 10 * random.randint(5,25)
        from_x_to_y_lower_bound = int(0.6 * from_x_to_y_average)
        from_x_to_y_upper_bound = int(1.4 * from_x_to_y_average)
        
        # print('from ' + str(x) + ' to ' + str(y) + ' average is ' + str(from_x_to_y_average))
        
        for z in range(0, total_nights + 1):
            from_x_to_y.append(random.randint(from_x_to_y_lower_bound, from_x_to_y_upper_bound))
        
        from_x.append(from_x_to_y)
    
    flight_fares.append(from_x)

# NOTE: flight_fare[x][x][z] doesn't make sense because it stands for the price from x to x.
# We could set all such values to be infinity, but here we choose to ignore them since they won't cause any problems. 

print("\nfrom 2 to 3 prices")
print(flight_fares[2][3])           


from 2 to 3 prices
[259, 311, 289, 228, 179, 271, 216, 260, 159, 321, 258, 230, 155, 266, 290, 232, 312, 283, 231, 221, 266, 169, 208, 141, 158, 148, 296, 250, 240, 278, 242, 321, 149, 174, 306, 182, 230, 193]


In [24]:
import numpy
# use a genetic algorithm to optimize the order of visiting the destinations

def compute_fitness(solution):
    # the fitness function for the genetic algorithm
    # the fitness is the cost of a given solution
    cost = 0
    day = 0
    for i in range(0, len(solution)-1):
        day += num_of_nights_to_stay[solution[i]]
        cost += flight_fares[solution[i]][solution[i+1]][day]
    # add the cost of returning to the start from the last destination
    day += num_of_nights_to_stay[solution[-1]]
    cost += flight_fares[solution[-1]][solution[0]][day]
    
    return cost


def generate_random_solution():
    # generate a random solution that represents the order of visiting the destinations
    # for example, [3, 1, 2, 0] means visiting in the order destinations[3] -> [1] -> [2] -> [0]
    random_solution = list(range(0, len(destinations)))
    random.shuffle(random_solution)
    return tuple(random_solution)
    

def mutate_solution(solution, max_mutations):
    # apply up to max_mutations mutations to each solution
    # for each mutation, randomly swap the position of two elements in a given solution
    solution = list(solution)
    num_of_mutations = random.randint(0, max_mutations)
    
    for i in range(max_mutations):
        index1 = random.randint(0, len(solution)-1)
        index2 = random.randint(0, len(solution)-1)
        while index1 == index2:
            index2 = random.randint(0, len(solution)-1)
        solution[index1], solution[index2] = solution[index2], solution[index1]
    return tuple(solution)


def generate_random_population(population_size):
    # generate a population of random solutions
    population = []
    for i in range(0, population_size):
        population.append(generate_random_solution())
    return population


def run(generations, population_size):
    
    # generations and population_size should be multiples of 10
    population_subset_size = int(population_size / 10.)
    generations_10pct = int(generations / 10.)
    
    # generate a population of random solutions
    population = generate_random_population(population_size)
    
    for generation in range(0, generations):
        
        # compute the fitness of each solution in the current population
        population_fitness = {}
        for solution in population:
            if solution in population_fitness:
                continue
            population_fitness[solution] = compute_fitness(solution)
    
        # take the top 10% solutions and produce offspring from them
        new_population = []
        for rank, solution in enumerate(sorted(population_fitness,
                                                   key=population_fitness.get)[:population_subset_size]):
            
            # show the current optimal solution
            if (generation % generations_10pct == 0 or generation == generations - 1) and rank == 0:
                print("Generation %d best: %d | Unique solutions: %d" % (generation,
                                                                       population_fitness[solution],
                                                                       len(population_fitness)))
                print(tuple([destinations[i] for i in solution]))
                print("")
            
            # the top 10% themselves should be included in the new population
            new_population.append(solution)
            
            # produce 9 offspring with up to 3 mutations for each solution
            for i in range(0, 9):
                new_population.append(mutate_solution(solution, 3))
            
        # Replace the old population with the new population of offspring 
        for i in range(len(population))[::-1]:
            del population[i]
        population = new_population

In [29]:
run(100, 100)

Generation 0 best: 1399 | Unique solutions: 100
('dest_c', 'dest_i', 'dest_m', 'dest_d', 'dest_l', 'dest_k', 'dest_f', 'dest_g', 'dest_h', 'dest_e', 'dest_n', 'dest_o', 'dest_a', 'dest_b', 'dest_j')

Generation 10 best: 1316 | Unique solutions: 100
('dest_i', 'dest_o', 'dest_j', 'dest_n', 'dest_e', 'dest_a', 'dest_b', 'dest_m', 'dest_l', 'dest_d', 'dest_k', 'dest_h', 'dest_f', 'dest_g', 'dest_c')

Generation 20 best: 1246 | Unique solutions: 100
('dest_i', 'dest_a', 'dest_d', 'dest_k', 'dest_h', 'dest_o', 'dest_b', 'dest_l', 'dest_m', 'dest_j', 'dest_n', 'dest_e', 'dest_f', 'dest_g', 'dest_c')

Generation 30 best: 1165 | Unique solutions: 100
('dest_c', 'dest_i', 'dest_g', 'dest_h', 'dest_n', 'dest_k', 'dest_e', 'dest_o', 'dest_f', 'dest_j', 'dest_a', 'dest_l', 'dest_d', 'dest_m', 'dest_b')

Generation 40 best: 1165 | Unique solutions: 99
('dest_c', 'dest_i', 'dest_g', 'dest_h', 'dest_n', 'dest_k', 'dest_e', 'dest_o', 'dest_f', 'dest_j', 'dest_a', 'dest_l', 'dest_d', 'dest_m', 'dest_b'